In [ ]:
#Set Path to mapped_links. This file is precalculated by apping the links to each other.
#Set target path name for the output as link_ids_near_precip_csv     
#(This is the output, this will all the links mapped with the given maximum perpendicular distance)

In [20]:
import pandas as pd
import numpy as np
from numpy import ones,vstack
from numpy.linalg import lstsq
import math
import csv
import geopy.distance
import geopy.point

In [24]:
mapped_links_csv_file = "mapped_links.csv"
link_ids_near_precip_csv = "/home/gayan/Desktop/CMLARE/Data/mapped_links_near_Orugodawatta"
mapped_links_field_names = ["cell_tower_id1","cell_tower_id2","source1_name","source2_name", "district1","district2",
                            "latitude1","latitude2","longitude1","longitude2","nearest_precip_station","distance",
                            "nearest_precip_lat","nearest_precip_long","id_1","id_2"]
formatted_links_field_names = ["cell_tower_id1","cell_tower_id2","source1_name","source2_name", "district1","district2",
                            "latitude1","latitude2","longitude1","longitude2","nearest_precip_station","distance",
                            "nearest_precip_lat","nearest_precip_long","id_1","id_2","PathLength"]
mapped_links_near_precip = []
precip_devices_csv_file = "precipitaion_stations_orugodawatta.csv"
precip_devices_field_names = ["name","lat","long","threshold"]
# maximum_perpendicular_distance = 10
link_ids_near_precip_csv = link_ids_near_precip_csv + ".csv"

In [25]:
def calc_equation(lat1, lang1, lat2, lang2):
    points = [(lat1, lang1), (lat2,lang2)]
    x_coords, y_coords = zip(*points)
    A = vstack([x_coords, ones(len(x_coords))]).T
    m, c = lstsq(A, y_coords)[0]
    #print("Line Solution is y = {m}x + {c}".format(m=m, c=c))
    return m,c

def calc_distance(lat1, lang1, lat2, lang2,lat_precip,long_precip):
#     m,c = calc_equation(lat1, lang1, lat2, lang2)
#     long = np.array([long_precip])
#     lat = np.array([lat_precip])
#     distance_function = lambda lat,long : abs(lat*m - long + c)/math.sqrt(lat**2 + long**2)
#     vec_distance_function = np.vectorize(distance_function)
#     distances = vec_distance_function(lat,long)*1000000
#     index_of_minimum_distance_precip = np.argmin(distances)
    lat1= float(lat1)
    lat2 = float(lat2)
    lang1 = float(lang1)
    lang2 = float(lang2)
    start_cord = (lat_precip,long_precip)
    mid_point = ((lat1+lat2)/2,(lang1+lang2)/2)
    print(start_cord)
    print(mid_point)
    distance = geopy.distance.vincenty(start_cord, mid_point).km
    print(distance)
    return distance
def path_distance(lat1,lang1,lat2,lang2):
    lat1= float(lat1)
    lat2 = float(lat2)
    lang1 = float(lang1)
    lang2 = float(lang2)
    start = (lat1,lang1)
    end = (lat2,lang2)
    return geopy.distance.vincenty(start, end).km    

In [26]:
precip_devices = []
with open(precip_devices_csv_file,"r") as precip_devices_csv:
    precip_devices_dict = csv.DictReader(precip_devices_csv,fieldnames=precip_devices_field_names)
    next(precip_devices_dict)
    for precip_device in precip_devices_dict:
        prep = {}
        prep['name'] = precip_device['name']
        prep['lat'] = float(precip_device['lat'])
        prep['long'] = float(precip_device['long'])
        prep["threshold"] = float(precip_device["threshold"])
        precip_devices.append(prep)
with open(mapped_links_csv_file,"r") as mapped_links_csv:
    mapped_links = csv.DictReader(mapped_links_csv, fieldnames=mapped_links_field_names)
    next(mapped_links)
#     print(precip_devices)
    for link in mapped_links:
        for precip_device in precip_devices:
            distance = calc_distance(link["latitude1"],link["longitude1"],link["latitude2"],link["longitude2"],precip_device["lat"],precip_device["long"])
            if(distance<=precip_device["threshold"]):
#                 print(precip_device["threshold"])
#                 print("***************")
                link["distance"] = distance
                link["nearest_precip_station"] = precip_device["name"]
                link["nearest_precip_lat"] = precip_device["lat"]
                link["nearest_precip_long"] = precip_device["long"]
                link["PathLength"] = path_distance(link["latitude1"],link["longitude1"],link["latitude2"],link["longitude2"])
                mapped_links_near_precip.append(link)
                break
    with open(link_ids_near_precip_csv,"w") as processed_file_csv:
        dictionary_writer = csv.DictWriter(processed_file_csv,formatted_links_field_names)
        dictionary_writer.writeheader()
        mapped_links_near_precip = [dict(t) for t in set([tuple(d.items()) for d in mapped_links_near_precip])]
        mapped_links_near_precip_sorted = sorted(mapped_links_near_precip , key=lambda k: k['nearest_precip_station'])
        dictionary_writer.writerows(mapped_links_near_precip_sorted)

(6.943741, 79.87887)
(9.2707405, 80.78805)
276.17504873915425
(6.943741, 79.87887)
(8.948889, 80.7344445)
240.9888474833337
(6.943741, 79.87887)
(8.948889, 80.7344445)
240.9888474833337
(6.943741, 79.87887)
(7.315825, 79.97370000000001)
42.461701047478556
(6.943741, 79.87887)
(9.6944795, 80.21310550000001)
306.44477712756134
(6.943741, 79.87887)
(6.97303, 79.905)
4.339275912606597
(6.943741, 79.87887)
(6.97303, 79.905)
4.339275912606597
(6.943741, 79.87887)
(6.911635, 79.9141)
5.269250021833673
(6.943741, 79.87887)
(6.911635, 79.9141)
5.269250021833673
(6.943741, 79.87887)
(7.3337995, 80.38635300000001)
70.73377343768219
(6.943741, 79.87887)
(6.900985, 79.91005)
5.850759533096737
(6.943741, 79.87887)
(8.577917, 81.233472)
234.49710070507396
(6.943741, 79.87887)
(7.110075, 79.88685)
18.41614458406674
(6.943741, 79.87887)
(8.572083500000002, 81.23166649999999)
233.87377754831184
(6.943741, 79.87887)
(8.572083500000002, 81.23166649999999)
233.87377754831184
(6.943741, 79.87887)
(7.7808435

(9.1225, 80.600833)
253.76324468456028
(6.943741, 79.87887)
(6.945595, 80.02605)
16.26588761804047
(6.943741, 79.87887)
(6.945595, 80.02605)
16.26588761804047
(6.943741, 79.87887)
(9.0369445, 80.58527749999999)
244.24665457961964
(6.943741, 79.87887)
(9.0369445, 80.58527749999999)
244.24665457961964
(6.943741, 79.87887)
(7.5323515, 80.3783835)
85.32658844221295
(6.943741, 79.87887)
(8.35125, 81.043611)
201.85470082768992
(6.943741, 79.87887)
(7.915075, 80.69579999999999)
140.25752848554063
(6.943741, 79.87887)
(7.915075, 80.69579999999999)
140.25752848554063
(6.943741, 79.87887)
(7.581555, 80.07130000000001)
73.668632873196
(6.943741, 79.87887)
(7.581555, 80.07130000000001)
73.668632873196
(6.943741, 79.87887)
(7.698095, 80.148889)
88.59331894072069
(6.943741, 79.87887)
(7.698095, 80.148889)
88.59331894072069
(6.943741, 79.87887)
(6.8680105, 79.9039)
8.820077249649234
(6.943741, 79.87887)
(6.8680105, 79.9039)
8.820077249649234
(6.943741, 79.87887)
(9.1272415, 80.684147)
257.28325776285

(7.1536605, 80.128161)
36.021378854280826
(6.943741, 79.87887)
(9.82015, 80.2346)
320.5283706734228
(6.943741, 79.87887)
(9.82015, 80.2346)
320.5283706734228
(6.943741, 79.87887)
(6.8812585, 79.96115)
11.420788230634543
(6.943741, 79.87887)
(6.8812585, 79.96115)
11.420788230634543
(6.943741, 79.87887)
(6.91284, 79.89755)
3.992482323847766
(6.943741, 79.87887)
(6.91284, 79.89755)
3.992482323847766
(6.943741, 79.87887)
(6.980522000000001, 79.87925)
4.067849294522861
(6.943741, 79.87887)
(6.980522000000001, 79.87925)
4.067849294522861
(6.943741, 79.87887)
(6.9536999999999995, 79.86940000000001)
1.5192721799105258
(6.943741, 79.87887)
(6.9536999999999995, 79.86940000000001)
1.5192721799105258
(6.943741, 79.87887)
(7.471135, 80.43674999999999)
84.84274693975412
(6.943741, 79.87887)
(7.471135, 80.43674999999999)
84.84274693975412
(6.943741, 79.87887)
(8.3731945, 79.7755555)
158.4994655507343
(6.943741, 79.87887)
(8.3731945, 79.7755555)
158.4994655507343
(6.943741, 79.87887)
(7.272805, 80.150

(6.943741, 79.87887)
(8.930972, 79.99375)
220.14359666048165
(6.943741, 79.87887)
(7.0789349999999995, 80.04195)
23.414260479799378
(6.943741, 79.87887)
(7.0789349999999995, 80.04195)
23.414260479799378
(6.943741, 79.87887)
(9.657695, 80.1169165)
301.29996824476336
(6.943741, 79.87887)
(9.657695, 80.1169165)
301.29996824476336
(6.943741, 79.87887)
(8.8857225, 80.8445835)
239.72066098056783
(6.943741, 79.87887)
(9.755343, 79.950928)
311.0585511859767
(6.943741, 79.87887)
(7.1762505, 80.096961)
35.23843135909713
(6.943741, 79.87887)
(7.1762505, 80.096961)
35.23843135909713
(6.943741, 79.87887)
(7.078453, 79.999961)
20.024060170211904
(6.943741, 79.87887)
(7.286912, 80.2093165)
52.65787303621544
(6.943741, 79.87887)
(7.8658395, 80.648678)
132.7464702100086
(6.943741, 79.87887)
(7.8658395, 80.648678)
132.7464702100086
(6.943741, 79.87887)
(8.072986, 79.7248835)
126.0380162101693
(6.943741, 79.87887)
(8.072986, 79.7248835)
126.0380162101693
(6.943741, 79.87887)
(6.859571, 79.8767445)
9.3113

(9.343384, 80.525789)
274.800291318777
(6.943741, 79.87887)
(7.6989285, 80.1380835)
88.28613910270539
(6.943741, 79.87887)
(8.454999999999998, 80.21486150000001)
171.19740034305536
(6.943741, 79.87887)
(6.8755555, 79.86236099999999)
7.758245779901026
(6.943741, 79.87887)
(7.1656365, 80.0342945)
29.950958904223853
(6.943741, 79.87887)
(7.1656365, 80.0342945)
29.950958904223853
(6.943741, 79.87887)
(7.43032, 80.00125)
55.48326829886052
(6.943741, 79.87887)
(7.43032, 80.00125)
55.48326829886052
(6.943741, 79.87887)
(6.898142, 79.8741835)
5.069338529487691
(6.943741, 79.87887)
(6.898142, 79.8741835)
5.069338529487691
(6.943741, 79.87887)
(6.898142, 79.8741835)
5.069338529487691
(6.943741, 79.87887)
(6.8793055, 79.8865275)
7.1760207076318085
(6.943741, 79.87887)
(6.8793055, 79.8865275)
7.1760207076318085
(6.943741, 79.87887)
(7.482065, 80.1030945)
64.47937977518076
(6.943741, 79.87887)
(6.8975865, 79.875989)
5.1141651211523484
(6.943741, 79.87887)
(8.75769, 80.4998)
211.9781655147685
(6.943